## Draw point cloud with normals and light vectors at each point

In [37]:
import raster_relight as rr
import json
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt

In [3]:
# Recreate the raster rendering function step-by-step

In [4]:
config = rr.parse_config()

Reading config from config.ini


In [7]:
with open(config['paths']['transforms_file'], 'r') as tf:
    image_transforms = json.loads(tf.read())

In [8]:
image_number = int(config['images']['image_number'])

In [10]:
# Loading Images
W, H, images, image_normal, image_rgbd = rr.load_images(
    image_number,
    channels=['albedo', 'depth', 'normal'])

Bit depth used for depth remapping was 8


In [11]:
# Loading Lights and Light Transforms
with open(config['paths']['lights_file'], 'r') as lf:
    lights_info = json.loads(lf.read())

In [12]:
print(f"Loaded {config['paths']['lights_file']}. Number of lights is {len(lights_info['lights'])}")

Loaded /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/lights_intrinsic_tester_sphere.json. Number of lights is 9


In [17]:
light_transforms = {light['name_light'].lower(): np.array(light['transformation_matrix']) for light in lights_info['lights']}
light_locations = {name: rr.to_translation(transform) for (name, transform) in light_transforms.items()}

In [20]:
# Compute Camera Parameters
intrinsics, c2w, R, T = rr.get_camera_parameters(W, H,
                                              image_transforms["camera_angle_x"],
                                              image_number,
                                              image_transforms)
rr.check_rotation(R)

det(R) = 1.0000000311834887
R^T @ R =
[[ 1.00000007e+00 -2.90236812e-08  8.29079916e-09]
 [-2.90236812e-08  1.00000000e+00 -3.13000683e-08]
 [ 8.29079916e-09 -3.13000683e-08  9.99999986e-01]]


In [21]:
print("Using the following intrinsics matrix:")
print("K = ")
print(intrinsics.intrinsic_matrix)

Using the following intrinsics matrix:
K = 
[[1.11111103e+03 0.00000000e+00 4.00000000e+02]
 [0.00000000e+00 1.11111103e+03 4.00000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [23]:
# ## Creating Point Cloud
occupied_mask = rr.get_occupancy(np.asarray(images['depth']))
print(f"There are {occupied_mask.sum()} points with finite depth in the image.")

There are 263345 points with finite depth in the image.


In [31]:
# ### Using Open3D RGBD + Point Cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    image_rgbd,
    o3d.camera.PinholeCameraIntrinsic(intrinsics)
)

# Flip it, otherwise the pointcloud will be upside down
pcd.transform(
    [[1, 0, 0, 0],
     [0, -1, 0, 0],
     [0, 0, -1, 0],
     [0, 0, 0, 1]])

# Pose the point cloud with the camera transform
pcd.transform(c2w)


PointCloud with 640000 points.

In [35]:
o3d.visualization.draw_geometries([pcd])

1.9764706

In [ ]:
# Get Normals
camera_normals = get_camera_space_normals(image_normal) * occupied_mask[..., np.newaxis]
world_normals = get_world_space_normals(camera_normals, R)